In [4]:
import numpy as np
import scipy as sp
import scipy.linalg
import matplotlib.pyplot as plt
import time
from scipy.optimize import minimize
import multiprocessing as mp
from MD_Parser import *
from kern_help import *
import math

In [5]:
%load_ext Cython

### Get example data.

In [6]:
outfile ='/Users/jonpvandermause/Research/GP/Datasets/SiC_MD/sic_md.out'
Si_MD_Parsed = parse_qe_pwscf_md_output(outfile)

# set crystal structure
dim = 3
alat = 4.344404578
unit_cell = [[0.0, alat/2, alat/2], [alat/2, 0.0, alat/2], \
                    [alat/2, alat/2, 0.0]] # fcc primitive cell
unit_pos = [['Si',[0,0,0]],['Si',[alat/4, alat/4, alat/4]]]
brav_mat = np.array([[0.0, alat/2, alat/2], [alat/2, 0.0, alat/2], \
                    [alat/2, alat/2, 0.0]])*dim
brav_inv = np.linalg.inv(brav_mat)

# bravais vectors
vec1 = brav_mat[:,0].reshape(3,1)
vec2 = brav_mat[:,1].reshape(3,1)
vec3 = brav_mat[:,2].reshape(3,1)

# build force field from single snapshot
cutoff = 3.6
pos = Si_MD_Parsed[1]['positions']
typs = Si_MD_Parsed[1]['elements']
fcs = fc_conv(Si_MD_Parsed[2]['forces'])

### Test covariance functions.

In [7]:
noa = len(pos)
time1 = time.time()
envs = get_envs(pos, typs, brav_mat, brav_inv, vec1, vec2, vec3, cutoff)
time2 = time.time()
print('it takes %.2f s to create %i atomic environments, or about %.2f s per environment.' % (time2-time1,noa,(time2-time1)/noa))

it takes 3.64 s to create 54 atomic environments, or about 0.07 s per environment.


In [8]:
x1 = envs[0]
x2 = envs[1]
d1 = 'xrel'
d2 = 'yrel'
sig = 1
ls = 1

time1 = time.time()
two_body(x1, x2, d1, d2, sig, ls)
time2 = time.time()
print('it takes %.3f us to calculate the two body kernel between two chemical environments.\n\
 the first chemical environment has %i atoms.\n\
 the second chemical environment has %i atoms.\n\
 there are %i possible distances compared, so it takes about %.3f us per distance.'
      % ((time2-time1)*1e6,len(x1['dists']),len(x2['dists']),len(x1['dists'])*len(x2['dists']),\
        1e6*(time2-time1)/(len(x1['dists'])*len(x2['dists']))))

it takes 443.935 us to calculate the two body kernel between two chemical environments.
 the first chemical environment has 23 atoms.
 the second chemical environment has 24 atoms.
 there are 552 possible distances compared, so it takes about 0.804 us per distance.


In [9]:
# get number of triplets in chemical environment
def count_trips(x1):
    count = 0
    for n in x1['trip_dict']['dists']:
        count+=len(n)
    return count

In [10]:
time1 = time.time()
three_body(x1, x2, d1, d2, sig, ls)
time2 = time.time()
print('it takes %.3f s to calculate the three body kernel between two chemical environments.\n\
 the first chemical environment has %i atoms and %i triplets.\n\
 the second chemical environment has %i atoms and %i triplets.\n\
 there are %i possible triplet comparisons, so it takes about %.3f us per triplet.'
      % ((time2-time1),len(x1['dists']),count_trips(x1),len(x2['dists']),count_trips(x2),\
        count_trips(x1)*count_trips(x2),1e6*(time2-time1)/(count_trips(x1)*count_trips(x2))))

it takes 0.132 s to calculate the three body kernel between two chemical environments.
 the first chemical environment has 23 atoms and 276 triplets.
 the second chemical environment has 24 atoms and 300 triplets.
 there are 82800 possible triplet comparisons, so it takes about 1.593 us per triplet.


### Cythonize the k3 function.

In [8]:
# two body kernel
def k2(ri,ci,rj,cj,d,e,f):
    rr = (ri-rj)*(ri-rj)
    k2 = d*math.exp(-f*rr)*ci*cj*(e-rr)
    return k2

# trip type 1: all atoms the same
def k3dt1(ri1,ri2,ri3,ci1,ci2,ci3,\
             rj1,rj2,rj3,cj1,cj2,cj3,\
             d,e,f):
    
    # define quantities that will reappear
    rsum = ri1*ri1+ri2*ri2+ri3*ri3+rj1*rj1+rj2*rj2+rj3*rj3
    
    r11 = ri1-rj1
    r12 = ri1-rj2
    r13 = ri1-rj3
    r21 = ri2-rj1
    r22 = ri2-rj2
    r23 = ri2-rj3
    r31 = ri3-rj1
    r32 = ri3-rj2
    r33 = ri3-rj3
    
    rr11 = ri1*rj1
    rr12 = ri1*rj2
    rr13 = ri1*rj3
    rr21 = ri2*rj1
    rr22 = ri2*rj2
    rr23 = ri2*rj3
    rr31 = ri3*rj1
    rr32 = ri3*rj2
    rr33 = ri3*rj3
    
    cc11 = ci1*cj1
    cc12 = ci1*cj2
    cc13 = ci1*cj3
    cc21 = ci2*cj1
    cc22 = ci2*cj2
    cc23 = ci2*cj3
    cc31 = ci3*cj1
    cc32 = ci3*cj2
    cc33 = ci3*cj3
    
    rci11 = r11*ci1
    rci12 = r12*ci1
    rci13 = r13*ci1
    rci21 = r21*ci2
    rci22 = r22*ci2
    rci23 = r23*ci2
    rci31 = r31*ci3
    rci32 = r32*ci3
    rci33 = r33*ci3
    
    rcj11 = r11*cj1
    rcj12 = r12*cj2
    rcj13 = r13*cj3
    rcj21 = r21*cj1
    rcj22 = r22*cj2
    rcj23 = r23*cj3
    rcj31 = r31*cj1
    rcj32 = r32*cj2
    rcj33 = r33*cj3
    
    # sum over six permutations
    derv = d*math.exp(-f*rsum)*\
        ((math.exp(2*f*(rr11+rr22+rr33))*(e*(cc11+cc22+cc33)-(rci11+rci22+rci33)*(rcj11+rcj22+rcj33)))+
        (math.exp(2*f*(rr11+rr23+rr32))*(e*(cc11+cc23+cc32)-(rci11+rci23+rci32)*(rcj11+rcj23+rcj32)))+
        (math.exp(2*f*(rr12+rr21+rr33))*(e*(cc12+cc21+cc33)-(rci12+rci21+rci33)*(rcj12+rcj21+rcj33)))+
        (math.exp(2*f*(rr12+rr23+rr31))*(e*(cc12+cc23+cc31)-(rci12+rci23+rci31)*(rcj12+rcj23+rcj31)))+
        (math.exp(2*f*(rr13+rr21+rr32))*(e*(cc13+cc21+cc32)-(rci13+rci21+rci32)*(rcj13+rcj21+rcj32)))+
        (math.exp(2*f*(rr13+rr22+rr31))*(e*(cc13+cc22+cc31)-(rci13+rci22+rci31)*(rcj13+rcj22+rcj31))))

    return derv

# triplet type 2: two different atoms
def k3dt2(ri1,ri2,ri3,ci1,ci2,ci3,\
             rj1,rj2,rj3,cj1,cj2,cj3,\
             d,e,f):
    
    # define quantities that will reappear
    rsum = ri1*ri1+ri2*ri2+ri3*ri3+rj1*rj1+rj2*rj2+rj3*rj3
    
    r11 = ri1-rj1
    r12 = ri1-rj2
    r21 = ri2-rj1
    r22 = ri2-rj2
    
    r33 = ri3-rj3
    rr33 = ri3*rj3
    cc33 = ci3*cj3
    rci33 = r33*ci3
    rcj33 = r33*cj3
    
    # sum over permutations
    derv = d*math.exp(-f*rsum)*\
        ((math.exp(2*f*(ri1*rj1+ri2*rj2+rr33))*(e*(ci1*cj1+ci2*cj2+cc33)-(r11*ci1+r22*ci2+rci33)*(r11*cj1+r22*cj2+rcj33)))+\
        (math.exp(2*f*(ri1*rj2+ri2*rj1+rr33))*(e*(ci1*cj2+ci2*cj1+cc33)-(r12*ci1+r21*ci2+rci33)*(r12*cj2+r21*cj1+rcj33))))

    return derv

# triplet type 3: three different atoms
def k3dt3(ri1,ri2,ri3,ci1,ci2,ci3,\
             rj1,rj2,rj3,cj1,cj2,cj3,\
             d,e,f):
    
    r1 = ri1-rj1
    r2 = ri2-rj2
    r3 = ri3-rj3
    derv = (e*(ci1*cj1+ci2*cj2+ci3*cj3)-(r1*ci1+r2*ci2+r3*ci3)*(r1*cj1+r2*cj2+r3*cj3))*\
            d*math.exp(-f*(r1*r1+r2*r2+r3*r3))

    return derv

def k3(typ,ri1,ri2,ri3,ci1,ci2,ci3,\
             rj1,rj2,rj3,cj1,cj2,cj3,\
             d,e,f):
    if typ==1:
        k3 = k3dt1(ri1,ri2,ri3,ci1,ci2,ci3,\
             rj1,rj2,rj3,cj1,cj2,cj3,\
             d,e,f)
    if typ==2:
        k3 = k3dt2(ri1,ri2,ri3,ci1,ci2,ci3,\
             rj1,rj2,rj3,cj1,cj2,cj3,\
             d,e,f)
    if typ==3:
        k3 = k3dt3(ri1,ri2,ri3,ci1,ci2,ci3,\
             rj1,rj2,rj3,cj1,cj2,cj3,\
             d,e,f)
        
    return k3

In [15]:
%%cython -a
cimport cython
from libc.math cimport exp

# two body kernel
cdef k2_c(float ri,float ci,float rj,float cj,float d,float e,float f):
    cdef float rr = (ri-rj)*(ri-rj)
    cdef float k2 = d*exp(-f*rr)*ci*cj*(e-rr)
    return k2

# triplet type 3: three different atoms
cdef k3dt3_c(float ri1,float ri2,float ri3,float ci1,float ci2,float ci3,\
             float rj1,float rj2,float rj3,float cj1,float cj2,float cj3,\
             float d,float e,float f):
    
    cdef float r1 = ri1-rj1
    cdef float r2 = ri2-rj2
    cdef float r3 = ri3-rj3
    return (e*(ci1*cj1+ci2*cj2+ci3*cj3)-(r1*ci1+r2*ci2+r3*ci3)*(r1*cj1+r2*cj2+r3*cj3))*\
            d*exp(-f*(r1*r1+r2*r2+r3*r3))
    
# triplet type 2: two different atoms
cdef k3dt2_c(float ri1,float ri2,float ri3,float ci1,float ci2,float ci3,\
             float rj1,float rj2,float rj3,float cj1,float cj2,float cj3,\
             float d,float e,float f):
    
    # define quantities that will reappear
    cdef float rsum = ri1*ri1+ri2*ri2+ri3*ri3+rj1*rj1+rj2*rj2+rj3*rj3
    
    cdef float r11 = ri1-rj1
    cdef float r12 = ri1-rj2
    cdef float r21 = ri2-rj1
    cdef float r22 = ri2-rj2
    
    cdef float r33 = ri3-rj3
    cdef float rr33 = ri3*rj3
    cdef float cc33 = ci3*cj3
    cdef float rci33 = r33*ci3
    cdef float rcj33 = r33*cj3
    
    # sum over permutations
    return d*exp(-f*rsum)*\
        ((exp(2*f*(ri1*rj1+ri2*rj2+rr33))*(e*(ci1*cj1+ci2*cj2+cc33)-(r11*ci1+r22*ci2+rci33)*(r11*cj1+r22*cj2+rcj33)))+\
        (exp(2*f*(ri1*rj2+ri2*rj1+rr33))*(e*(ci1*cj2+ci2*cj1+cc33)-(r12*ci1+r21*ci2+rci33)*(r12*cj2+r21*cj1+rcj33))))
    
# trip type 1: all atoms the same
cdef k3dt1_c(float ri1,float ri2,float ri3,float ci1,float ci2,float ci3,\
             float rj1,float rj2,float rj3,float cj1,float cj2,float cj3,\
             float d,float e,float f):
    
    # define quantities that will reappear
    cdef float rsum = ri1*ri1+ri2*ri2+ri3*ri3+rj1*rj1+rj2*rj2+rj3*rj3
    
    cdef float r11 = ri1-rj1
    cdef float r12 = ri1-rj2
    cdef float r13 = ri1-rj3
    cdef float r21 = ri2-rj1
    cdef float r22 = ri2-rj2
    cdef float r23 = ri2-rj3
    cdef float r31 = ri3-rj1
    cdef float r32 = ri3-rj2
    cdef float r33 = ri3-rj3
    
    cdef float rr11 = ri1*rj1
    cdef float rr12 = ri1*rj2
    cdef float rr13 = ri1*rj3
    cdef float rr21 = ri2*rj1
    cdef float rr22 = ri2*rj2
    cdef float rr23 = ri2*rj3
    cdef float rr31 = ri3*rj1
    cdef float rr32 = ri3*rj2
    cdef float rr33 = ri3*rj3
    
    cdef float cc11 = ci1*cj1
    cdef float cc12 = ci1*cj2
    cdef float cc13 = ci1*cj3
    cdef float cc21 = ci2*cj1
    cdef float cc22 = ci2*cj2
    cdef float cc23 = ci2*cj3
    cdef float cc31 = ci3*cj1
    cdef float cc32 = ci3*cj2
    cdef float cc33 = ci3*cj3
    
    cdef float rci11 = r11*ci1
    cdef float rci12 = r12*ci1
    cdef float rci13 = r13*ci1
    cdef float rci21 = r21*ci2
    cdef float rci22 = r22*ci2
    cdef float rci23 = r23*ci2
    cdef float rci31 = r31*ci3
    cdef float rci32 = r32*ci3
    cdef float rci33 = r33*ci3
    
    cdef float rcj11 = r11*cj1
    cdef float rcj12 = r12*cj2
    cdef float rcj13 = r13*cj3
    cdef float rcj21 = r21*cj1
    cdef float rcj22 = r22*cj2
    cdef float rcj23 = r23*cj3
    cdef float rcj31 = r31*cj1
    cdef float rcj32 = r32*cj2
    cdef float rcj33 = r33*cj3
    
    # sum over six permutations
    return d*exp(-f*rsum)*\
        ((exp(2*f*(rr11+rr22+rr33))*(e*(cc11+cc22+cc33)-(rci11+rci22+rci33)*(rcj11+rcj22+rcj33)))+
        (exp(2*f*(rr11+rr23+rr32))*(e*(cc11+cc23+cc32)-(rci11+rci23+rci32)*(rcj11+rcj23+rcj32)))+
        (exp(2*f*(rr12+rr21+rr33))*(e*(cc12+cc21+cc33)-(rci12+rci21+rci33)*(rcj12+rcj21+rcj33)))+
        (exp(2*f*(rr12+rr23+rr31))*(e*(cc12+cc23+cc31)-(rci12+rci23+rci31)*(rcj12+rcj23+rcj31)))+
        (exp(2*f*(rr13+rr21+rr32))*(e*(cc13+cc21+cc32)-(rci13+rci21+rci32)*(rcj13+rcj21+rcj32)))+
        (exp(2*f*(rr13+rr22+rr31))*(e*(cc13+cc22+cc31)-(rci13+rci22+rci31)*(rcj13+rcj22+rcj31))))
    
cdef k3_c(int typ,float ri1,float ri2,float ri3,float ci1,float ci2,float ci3,\
             float rj1,float rj2,float rj3,float cj1,float cj2,float cj3,\
             float d,float e,float f):
    
    cdef float k3
    
    if typ==1:
        k3 = k3dt1_c(ri1,ri2,ri3,ci1,ci2,ci3,\
             rj1,rj2,rj3,cj1,cj2,cj3,\
             d,e,f)
    if typ==2:
        k3 = k3dt2_c(ri1,ri2,ri3,ci1,ci2,ci3,\
             rj1,rj2,rj3,cj1,cj2,cj3,\
             d,e,f)
    if typ==3:
        k3 = k3dt3_c(ri1,ri2,ri3,ci1,ci2,ci3,\
             rj1,rj2,rj3,cj1,cj2,cj3,\
             d,e,f) 
    return k3

# get two body kernel between chemical environments
def two_body_c(dict x1,dict x2,str d1,str d2,float sig,float ls):
    cdef float d= sig*sig/(ls*ls*ls*ls)
    cdef float e= ls*ls
    cdef float f= 1/(2*ls*ls)
    cdef float kern = 0
    
    # record central atom types
    cdef str c1 = x1['central_atom']
    cdef str c2 = x2['central_atom']
    
    cdef int m, n
    
    cdef int x1_len = len(x1['dists'])
    cdef int x2_len = len(x2['dists'])
    
    cdef str e1, e2
    cdef float r1, coord1, r2, coord2
    
    for m in range(x1_len):
        e1 = x1['types'][m]
        r1 = x1['dists'][m]
        coord1 = x1[d1][m]
        for n in range(x2_len):
            e2 = x2['types'][n]
            r2 = x2['dists'][n]
            coord2 = x2[d2][n]
            
            # check that atom types match
            if (c1==c2 and e1==e2) or (c1==e2 and c2==e1):
                kern+=k2_c(r1,coord1,r2,coord2,d,e,f)
                
    return kern

# get two body kernel between chemical environments
@cython.boundscheck(False)
def two_body_full_c(dict x1,dict x2,str d1,str d2,float sig,float ls):
    cdef float d= sig*sig/(ls*ls*ls*ls)
    cdef float e= ls*ls
    cdef float f= 1/(2*ls*ls)
    cdef float kern = 0
    
    # record central atom types
    cdef str c1 = x1['central_atom']
    cdef str c2 = x2['central_atom']
    
    cdef int m, n
    
    cdef int x1_len = len(x1['dists'])
    cdef int x2_len = len(x2['dists'])
    
    cdef str e1, e2
    cdef float r1, coord1, r2, coord2, rr
    
    for m in range(x1_len):
        e1 = x1['types'][m]
        r1 = x1['dists'][m]
        coord1 = x1[d1][m]
        for n in range(x2_len):
            e2 = x2['types'][n]
            r2 = x2['dists'][n]
            coord2 = x2[d2][n]
            
            # check that atom types match
            if (c1==c2 and e1==e2) or (c1==e2 and c2==e1):
                rr = (r1-r2)*(r1-r2)
                kern += d*exp(-f*rr)*coord1*coord2*(e-rr)
                
    return kern

# get three body kernel between two chemical environments
@cython.boundscheck(False)
def three_body_c(dict x1,dict x2,str d1,str d2,float sig,float ls):
    cdef float d= sig*sig/(ls*ls*ls*ls)
    cdef float e= ls*ls
    cdef float f= 1/(2*ls*ls)
    cdef float kern = 0
    
    cdef int m, n, p, q, typ
    
    cdef list x1_labs = x1['trip_dict']['labs']
    cdef list x2_labs = x2['trip_dict']['labs']
    cdef int x1_lab_len = len(x1_labs)
    cdef int x2_lab_len = len(x2_labs)
    cdef list x1_lab, x2_lab
    
    cdef list x1_typs = x1['trip_dict']['typs']
    
    cdef float ri1, ri2, ri3, ci1, ci2, ci3, rj1, rj2, rj3, cj1, cj2, cj3, k3
    
    cdef list x1_dists = x1['trip_dict']['dists']
    cdef list x2_dists = x2['trip_dict']['dists']
    cdef list x1_coords = x1['trip_dict'][d1]
    cdef list x2_coords = x2['trip_dict'][d2]
    
    for m in range(x1_lab_len):
        x1_lab = x1_labs[m]

        for n in range(x2_lab_len):
            x2_lab = x2_labs[n]

            # check triplet type
            if x1_lab==x2_lab:
                # loop over tripets of the same type
                typ = x1_typs[m]

                # loop over triplets in environment 1
                for p in range(len(x1_dists[m])):
                    # set distances
                    ri1 = x1_dists[m][p][0]
                    ri2 = x1_dists[m][p][1]
                    ri3 = x1_dists[m][p][2]

                    # set coordinates
                    ci1 = x1_coords[m][p][0]
                    ci2 = x1_coords[m][p][1]
                    ci3 = x1_coords[m][p][2]

                    # loop over triplets in environment 2
                    for q in range(len(x2_dists[n])):
                        # set distances
                        rj1 = x2_dists[n][q][0]
                        rj2 = x2_dists[n][q][1]
                        rj3 = x2_dists[n][q][2]

                        # set coordinates
                        cj1 = x2_coords[n][q][0]
                        cj2 = x2_coords[n][q][1]
                        cj3 = x2_coords[n][q][2]

                        # add to kernel
                        if typ==1:
                            kern+=k3dt1_c(ri1,ri2,ri3,ci1,ci2,ci3,\
                                 rj1,rj2,rj3,cj1,cj2,cj3,\
                                 d,e,f)
                        if typ==2:
                            kern+=k3dt2_c(ri1,ri2,ri3,ci1,ci2,ci3,\
                                 rj1,rj2,rj3,cj1,cj2,cj3,\
                                 d,e,f)
                        if typ==3:
                            kern+=k3dt3_c(ri1,ri2,ri3,ci1,ci2,ci3,\
                                 rj1,rj2,rj3,cj1,cj2,cj3,\
                                 d,e,f) 
                        
    return kern

# get three body kernel between two chemical environments
@cython.boundscheck(False)
def three_body_full_c(dict x1,dict x2,str d1,str d2,float sig,float ls):
    cdef float d= sig*sig/(ls*ls*ls*ls)
    cdef float e= ls*ls
    cdef float f= 1/(2*ls*ls)
    cdef float kern = 0
    
    cdef int m, n, p, q, typ
    
    cdef list x1_labs = x1['trip_dict']['labs']
    cdef list x2_labs = x2['trip_dict']['labs']
    cdef int x1_lab_len = len(x1_labs)
    cdef int x2_lab_len = len(x2_labs)
    cdef list x1_lab, x2_lab
    
    cdef list x1_typs = x1['trip_dict']['typs']
    
    cdef float ri1, ri2, ri3, ci1, ci2, ci3, rj1, rj2, rj3, cj1, cj2, cj3, k3
    
    cdef list x1_dists = x1['trip_dict']['dists']
    cdef list x2_dists = x2['trip_dict']['dists']
    cdef list x1_coords = x1['trip_dict'][d1]
    cdef list x2_coords = x2['trip_dict'][d2]
    
    cdef float rsum, r11, r12, r13, r21, r22, r23, r31, r32, r33, rr11, rr12, rr13, rr21, rr22, rr23, rr31, rr32,\
                rr33, cc11, cc12, cc13, cc21, cc22, cc23, cc31, cc32, cc33, rci11, rci12, rci13, rci21, rci22, rci23,\
                rci31, rci32, rci33, rcj11, rcj12, rcj13, rcj21, rcj22, rcj23, rcj31, rcj32, rcj33
    
    for m in range(x1_lab_len):
        x1_lab = x1_labs[m]

        for n in range(x2_lab_len):
            x2_lab = x2_labs[n]

            # check triplet type
            if x1_lab==x2_lab:
                # loop over tripets of the same type
                typ = x1_typs[m]

                # loop over triplets in environment 1
                for p in range(len(x1_dists[m])):
                    # set distances
                    ri1 = x1_dists[m][p][0]
                    ri2 = x1_dists[m][p][1]
                    ri3 = x1_dists[m][p][2]

                    # set coordinates
                    ci1 = x1_coords[m][p][0]
                    ci2 = x1_coords[m][p][1]
                    ci3 = x1_coords[m][p][2]

                    # loop over triplets in environment 2
                    for q in range(len(x2_dists[n])):
                        # set distances
                        rj1 = x2_dists[n][q][0]
                        rj2 = x2_dists[n][q][1]
                        rj3 = x2_dists[n][q][2]

                        # set coordinates
                        cj1 = x2_coords[n][q][0]
                        cj2 = x2_coords[n][q][1]
                        cj3 = x2_coords[n][q][2]

                        # add to kernel
                        if typ==1:
                            rsum = ri1*ri1+ri2*ri2+ri3*ri3+rj1*rj1+rj2*rj2+rj3*rj3
                            r11 = ri1-rj1
                            r12 = ri1-rj2
                            r13 = ri1-rj3
                            r21 = ri2-rj1
                            r22 = ri2-rj2
                            r23 = ri2-rj3
                            r31 = ri3-rj1
                            r32 = ri3-rj2
                            r33 = ri3-rj3

                            rr11 = ri1*rj1
                            rr12 = ri1*rj2
                            rr13 = ri1*rj3
                            rr21 = ri2*rj1
                            rr22 = ri2*rj2
                            rr23 = ri2*rj3
                            rr31 = ri3*rj1
                            rr32 = ri3*rj2
                            rr33 = ri3*rj3

                            cc11 = ci1*cj1
                            cc12 = ci1*cj2
                            cc13 = ci1*cj3
                            cc21 = ci2*cj1
                            cc22 = ci2*cj2
                            cc23 = ci2*cj3
                            cc31 = ci3*cj1
                            cc32 = ci3*cj2
                            cc33 = ci3*cj3

                            rci11 = r11*ci1
                            rci12 = r12*ci1
                            rci13 = r13*ci1
                            rci21 = r21*ci2
                            rci22 = r22*ci2
                            rci23 = r23*ci2
                            rci31 = r31*ci3
                            rci32 = r32*ci3
                            rci33 = r33*ci3

                            rcj11 = r11*cj1
                            rcj12 = r12*cj2
                            rcj13 = r13*cj3
                            rcj21 = r21*cj1
                            rcj22 = r22*cj2
                            rcj23 = r23*cj3
                            rcj31 = r31*cj1
                            rcj32 = r32*cj2
                            rcj33 = r33*cj3
                            
                            kern+=d*exp(-f*rsum)*\
                                ((exp(2*f*(rr11+rr22+rr33))*(e*(cc11+cc22+cc33)-(rci11+rci22+rci33)*(rcj11+rcj22+rcj33)))+
                                (exp(2*f*(rr11+rr23+rr32))*(e*(cc11+cc23+cc32)-(rci11+rci23+rci32)*(rcj11+rcj23+rcj32)))+
                                (exp(2*f*(rr12+rr21+rr33))*(e*(cc12+cc21+cc33)-(rci12+rci21+rci33)*(rcj12+rcj21+rcj33)))+
                                (exp(2*f*(rr12+rr23+rr31))*(e*(cc12+cc23+cc31)-(rci12+rci23+rci31)*(rcj12+rcj23+rcj31)))+
                                (exp(2*f*(rr13+rr21+rr32))*(e*(cc13+cc21+cc32)-(rci13+rci21+rci32)*(rcj13+rcj21+rcj32)))+
                                (exp(2*f*(rr13+rr22+rr31))*(e*(cc13+cc22+cc31)-(rci13+rci22+rci31)*(rcj13+rcj22+rcj31))))
                        if typ==2:
                            
                            rsum = ri1*ri1+ri2*ri2+ri3*ri3+rj1*rj1+rj2*rj2+rj3*rj3
    
                            r11 = ri1-rj1
                            r12 = ri1-rj2
                            r21 = ri2-rj1
                            r22 = ri2-rj2

                            r33 = ri3-rj3
                            rr33 = ri3*rj3
                            cc33 = ci3*cj3
                            rci33 = r33*ci3
                            rcj33 = r33*cj3

                            # sum over permutations
                            kern+=d*exp(-f*rsum)*\
                                ((exp(2*f*(ri1*rj1+ri2*rj2+rr33))*(e*(ci1*cj1+ci2*cj2+cc33)-(r11*ci1+r22*ci2+rci33)*(r11*cj1+r22*cj2+rcj33)))+\
                                (exp(2*f*(ri1*rj2+ri2*rj1+rr33))*(e*(ci1*cj2+ci2*cj1+cc33)-(r12*ci1+r21*ci2+rci33)*(r12*cj2+r21*cj1+rcj33))))
                        if typ==3:
                            r11 = ri1-rj1
                            r22 = ri2-rj2
                            r33 = ri3-rj3
                            kern+=(e*(ci1*cj1+ci2*cj2+ci3*cj3)-(r11*ci1+r22*ci2+r33*ci3)*(r11*cj1+r22*cj2+r33*cj3))*\
                                    d*exp(-f*(r11*r11+r22*r22+r33*r33))
                        
    return kern

In [16]:
# time two-body function
sig = 1
ls = 1

x1 = envs[0]
x2 = envs[1]

d1 = 'xrel'
d2 = 'yrel'

print(two_body(x1, x2, d1, d2, sig, ls))
%timeit two_body(x1, x2, d1, d2, sig, ls)

print(two_body_c(x1, x2, d1, d2, sig, ls))
%timeit two_body_c(x1, x2, d1, d2, sig, ls)

print(two_body_full_c(x1, x2, d1, d2, sig, ls))
%timeit two_body_full_c(x1, x2, d1, d2, sig, ls)

3.1221254160101246
212 µs ± 2.59 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
3.1221258640289307
40.2 µs ± 56.4 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
3.122126340866089
36.3 µs ± 438 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [251]:
# time three-body function
sig = 1
ls = 2

x1 = envs[0]
x2 = envs[53]

d1 = 'xrel'
d2 = 'yrel'

print(three_body(x1, x2, d1, d2, sig, ls))
%timeit three_body(x1, x2, d1, d2, sig, ls)

print(three_body_full_c(x1, x2, d1, d2, sig, ls))
%timeit three_body_full_c(x1, x2, d1, d2, sig, ls)

-19.87217475269006
63.3 ms ± 1.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
-19.872173309326172
1.33 ms ± 2.71 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### Import and test cython kernels.

In [16]:
import cython_kernels

In [14]:
# time three-body function
sig = 1
ls = 2

x1 = envs[0]
x2 = envs[53]

d1 = 'xrel'
d2 = 'yrel'

print(three_body(x1, x2, d1, d2, sig, ls))
%timeit three_body(x1, x2, d1, d2, sig, ls)

print(three_body_c(x1, x2, d1, d2, sig, ls))
%timeit three_body_c(x1, x2, d1, d2, sig, ls)

-19.87217475269006
65.2 ms ± 2.82 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
-19.872173309326172
1.35 ms ± 3.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [2]:
three_body_c

<function cython_kernels.three_body_c>

In [3]:
two_body_c

<function cython_kernels.two_body_c>

In [15]:
three_body

<function kern_help.three_body>